In [1]:
!pip install -q -U kaggle_environments

     |████████████████████████████████| 1.9 MB 5.1 MB/s 
     |████████████████████████████████| 62 kB 1.3 MB/s 


In [2]:
import numpy as np
import pandas as pd
import random

import matplotlib.pyplot as plt
import seaborn as sns

from kaggle_environments import make, evaluate

In [34]:
agents_list = ["rock_agent.py",
               "paper_agent.py",
               "scissors_agent.py",
               "rp_agent.py",
               "ps_agent.py",
               "rs_agent.py",
               "random_agent.py",
               "copy_opponent.py",
               "counter_opponent.py",
               "lose_reaction.py",
               "statistical.py",
               "roll_agent.py"]

In [4]:
%%writefile rock_agent.py

#Агент "Только Камень"
#0 - rock
#1 - paper
#2 - scissors
def rock_agent(observation, configuration):
    return 0

Writing rock_agent.py


In [5]:
%%writefile paper_agent.py

#Агент "Только Бумага"
def paper_agent(observation, configuration):
    return 1

Writing paper_agent.py


In [6]:
%%writefile scissors_agent.py

#Агент "Только Ножницы"
def scissors_agent(observation, configuration):
    return 2

Writing scissors_agent.py


In [7]:
%%writefile rp_agent.py

import random

#Агент "Камень Бумага"
def rp_agent(observation, configuration):
    return random.randrange(0, 2)

Writing rp_agent.py


In [8]:
%%writefile ps_agent.py

import random

#Агент "Бумага Ножницы"
def rp_agent(observation, configuration):
    return random.randrange(1, 3)

Writing ps_agent.py


In [9]:
%%writefile rs_agent.py

import random

#Агент "Камень Ножницы"
def rs_agent(observation, configuration):
    signs = [0, 2]
    return signs[random.randrange(0, 2)]

Writing rs_agent.py


In [10]:
%%writefile random_agent.py

import random

#Рандомный агент
def random_agent(observation, configuration):
    return random.randrange(0, configuration.signs-1)

Writing random_agent.py


In [11]:
%%writefile copy_opponent.py

import random

# Копирует ход оппонента
def copy_opponent(observation, configuration):    
    if observation.step > 0:
        return observation.lastOpponentAction
    else:
        return random.randrange(0, configuration.signs)

Writing copy_opponent.py


In [12]:
%%writefile counter_opponent.py

import random

# Играем контрисходы в ответ
def counter_opponent(observation, configuration):
    if observation.step > 0:
        return (observation.lastOpponentAction + 1) % configuration.signs
    else:
        return random.randrange(0, configuration.signs)

Writing counter_opponent.py


In [13]:
%%writefile lose_reaction.py

import random

#Играем исход, который проиграл бы в прошлом раунде
def counter_opponent(observation, configuration):
    if observation.step > 0:
        return (observation.lastOpponentAction + 2) % configuration.signs
    else:
        return random.randrange(0, configuration.signs)

Writing lose_reaction.py


In [24]:
%%writefile statistical.py

# Играем контрисход против самого популярного хода

def statistical(observation, configuration):
    global action_histogram
    if observation.step == 0:
        action_histogram = {}
        return
    action = observation.lastOpponentAction
    if action not in action_histogram:
        action_histogram[action] = 0
    action_histogram[action] += 1
    mode_action = None
    mode_action_count = None
    for k, v in action_histogram.items():
        if mode_action_count is None or v > mode_action_count:
            mode_action = k
            mode_action_count = v
            continue

    return (mode_action + 1) % configuration.signs

Writing statistical.py


In [41]:
%%writefile roll_agent.py

# Играем контрисход против самого популярного хода со сменой тактики

def roll_agent(observation, configuration):
    global action_histogram
    global alt_mode

    if observation.step == 0:
        action_histogram = {}
        alt_mode = False
        return

    #Если после 20% игр счет нас не устраивает, то меняем тактику
    if not alt_mode:
        if observation.step > configuration.episodeSteps * 0.2:
            if observation.reward < observation.step * 0.1:
                alt_mode = True
    if alt_mode:
        return (observation.lastOpponentAction + 1) % configuration.signs
    
    action = observation.lastOpponentAction
    if action not in action_histogram:
        action_histogram[action] = 0
    action_histogram[action] += 1
    mode_action = None
    mode_action_count = None
    for k, v in action_histogram.items():
        if mode_action_count is None or v > mode_action_count:
            mode_action = k
            mode_action_count = v
            continue

    return (mode_action + 1) % configuration.signs

Overwriting roll_agent.py


In [46]:
scores_table_size = len(agents_list)
scores_table = np.zeros((scores_table_size, scores_table_size), dtype=int)

total_score = []
for i in range(scores_table_size):
    for j in range(i + 1, scores_table_size):
        res = evaluate(
                "rps", #environment to use - no need to change
                [agents_list[i], agents_list[j]], #agents to evaluate
                configuration={"episodeSteps": 1000, "tieRewardThreshold":100} #number of episodes 
                )
        scores_table[i][j] = res[0][0]
        scores_table[j][i] = res[0][1]
    total_score.append(sum(scores_table[i]))
    print(f"Агент {agents_list[i]} закончил работу")
                       

Агент rock_agent.py закончил работу
Агент paper_agent.py закончил работу
Агент scissors_agent.py закончил работу
Агент rp_agent.py закончил работу
Агент ps_agent.py закончил работу
Агент rs_agent.py закончил работу
Агент random_agent.py закончил работу
Агент copy_opponent.py закончил работу
Агент counter_opponent.py закончил работу
Агент lose_reaction.py закончил работу
Агент statistical.py закончил работу
Агент roll_agent.py закончил работу


In [47]:
df_scores_table = pd.DataFrame(scores_table, index = agents_list, columns = agents_list)
display(df_scores_table)

,rock_agent.py,paper_agent.py,scissors_agent.py,rp_agent.py,ps_agent.py,rs_agent.py,random_agent.py,copy_opponent.py,counter_opponent.py,lose_reaction.py,statistical.py,roll_agent.py
rock_agent.py,0,-999,999,-481,0,521,-492,0,-999,998,-998,-998
paper_agent.py,999,0,-999,541,-504,0,500,0,-998,999,-997,-997
scissors_agent.py,-999,999,0,0,501,-521,0,0,-997,999,-999,-999
rp_agent.py,481,-541,0,0,-272,271,0,0,-248,231,-172,-163
ps_agent.py,0,504,-501,272,0,-237,252,0,-218,233,-176,-384
rs_agent.py,-521,0,521,-271,237,0,-282,0,-232,278,-138,-196
random_agent.py,492,-500,0,0,-252,282,0,0,-262,237,0,-305
copy_opponent.py,0,0,0,0,0,0,0,0,999,500,-247,-200
counter_opponent.py,999,998,997,248,218,232,262,-999,0,-500,599,0
lose_reaction.py,-998,-999,-999,-231,-233,-278,-237,-500,500,0,-996,-996


In [48]:
temp_result = [(agent, score) for agent, score in zip(agents_list, total_score)]
result_table = sorted(temp_result, key=lambda x: x[1])

for record in result_table[::-1]:
  print(f"{record[0]:<20}: {record[1]:>4} баллов")

roll_agent.py       : 5636 баллов
statistical.py      : 3726 баллов
counter_opponent.py : 3054 баллов
copy_opponent.py    : 1052 баллов
ps_agent.py         : -255 баллов
random_agent.py     : -308 баллов
rp_agent.py         : -413 баллов
rs_agent.py         : -604 баллов
paper_agent.py      : -1456 баллов
scissors_agent.py   : -2016 баллов
rock_agent.py       : -2449 баллов
lose_reaction.py    : -5967 баллов
